In [29]:
import requests
import os
import json
import pandas as pd
FOURSQUARE_API_KEY = os.environ.get("FOURSQUARE_API_KEY")
YELP_API_KEY = os.environ.get("YELP_API_KEY")

In [39]:
import import_ipynb
import city_bikes

In [43]:
df_bike.head()  # This should display the first few rows of the DataFrame


,station_name,latitude,longitude,total_bikes
0,Summerhill Ave / Maclennan Ave,43.685924,-79.376304,11
1,Queen St E / Joseph Duggan Rd,43.667763,-79.308117,18
2,Victoria Park Ave / Danforth Ave,43.691468,-79.288619,18
3,Navy Wharf Crt / Bremner Blvd,43.640722,-79.391051,7
4,420 Wellington St W,43.643834,-79.396649,14


In [50]:
df_bike.shape

(852, 4)

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [75]:
# Foursquare endpoint for searching places
foursquare_url = 'https://api.foursquare.com/v3/places/search'
fs_headers = {
    "Accept": "application/json",
    "Authorization": FOURSQUARE_API_KEY
}

def foursquare_places(latitude, longitude, query, radius=1000):
    params = {
        "query": query,
        "ll": f"{latitude},{longitude}",
        "radius": radius,
        "limit": 10  # Limit to 10 results
    }
    
    response = requests.get(foursquare_url, headers=fs_headers, params=params)
    return response.json()  # Return JSON data if the request is successful

In [77]:
# Using Lat Long of Toronto
fs_latitude = 43.6532
fs_longitude = -79.3832
fs_restaurants = foursquare_places(fs_latitude, fs_longitude, "restaurant")

fs_results = fs_restaurants.get('results', []) 
fs_restaurants_df = pd.DataFrame(fs_results) 
fs_restaurants_df.head() 

,fsq_id,categories,chains,closed_bucket,distance,geocodes,link,location,name,related_places,timezone
0,537773d1498e74a75bb75c1e,"[{'id': 13028, 'name': 'Breakfast Spot', 'shor...",[],VeryLikelyOpen,98,"{'main': {'latitude': 43.653268, 'longitude': ...",/v3/places/537773d1498e74a75bb75c1e,"{'address': '483 Bay St', 'country': 'CA', 'cr...",Eggspectation Bell Trinity Square,{'parent': {'fsq_id': '4ae5df5af964a520c4a221e...,America/Toronto
1,4d306dd82748b60c62b6dba0,"[{'id': 13065, 'name': 'Restaurant', 'short_na...",[],VeryLikelyOpen,216,"{'main': {'latitude': 43.654458, 'longitude': ...",/v3/places/4d306dd82748b60c62b6dba0,"{'address': '220 Yonge St', 'address_extended'...",Trattoria Mercatto,{'parent': {'fsq_id': '4ad77a12f964a520260b21e...,America/Toronto
2,4ad4c060f964a5207cf720e3,"[{'id': 13338, 'name': 'Seafood Restaurant', '...",[],LikelyOpen,312,"{'main': {'latitude': 43.650598, 'longitude': ...",/v3/places/4ad4c060f964a5207cf720e3,"{'address': '56 Temperance St', 'country': 'CA...",John & Sons Oyster House,{},America/Toronto
3,57d0a3fb498e0fb29e892eed,"[{'id': 13343, 'name': 'South American Restaur...",[],LikelyOpen,298,"{'main': {'latitude': 43.651907, 'longitude': ...",/v3/places/57d0a3fb498e0fb29e892eed,"{'address': '176 Yonge St', 'country': 'CA', '...",Leña Restaurante,{},America/Toronto
4,5d782433f6e3190008b1a914,"[{'id': 13263, 'name': 'Japanese Restaurant', ...",[],VeryLikelyOpen,303,"{'main': {'latitude': 43.655187, 'longitude': ...",/v3/places/5d782433f6e3190008b1a914,"{'address': '157 Dundas St W', 'country': 'CA'...",Gyubee Japanese Grill,{},America/Toronto


In [94]:
fs_restaurants_df.dtypes

fsq_id            object
categories        object
chains            object
closed_bucket     object
distance           int64
geocodes          object
link              object
location          object
name              object
related_places    object
timezone          object
dtype: object

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [92]:
# Starting with an empty list
fs_poi_data = []

#Selecting first 5 stations from the df_bike 

fs_selected_stations = df_bike.head(5)

# Iterate through each bike station in df_bike dataframe from city_bikes
for index, row in fs_selected_stations.iterrows():
    latitude = row['latitude'] 
    longitude = row['longitude']
    station_name = row['station_name']
    
    # Query for restaurants, bars, and banks
    for query in ["restaurant", "bar", "bank"]:
        pois = foursquare_places(latitude, longitude, query, radius=50)
        
        if pois and 'results' in pois:
            for place in pois['results']:
                distance = place.get('distance', None)
                poi_info = {
                    'station_name': station_name,
                    'name': place.get('name', ''),
                    'latitude': latitude,
                    'longitude': longitude,
                    'category': query,
                    'distance': distance 
                }
                fs_poi_data.append(poi_info)  # Add to the empty list

Looks like there are no rating provided with foresquare places. Will make another call and modify the code.

Put your parsed results into a DataFrame

In [91]:
# Create a df from the data
fs_pois_df = pd.DataFrame(fs_poi_data)

# Check the first few rows of the new df
fs_pois_df.head()

,station_name,name,latitude,longitude,category,distance
0,Summerhill Ave / Maclennan Ave,Dolce Bakery,43.685924,-79.376304,restaurant,38
1,Summerhill Ave / Maclennan Ave,Ashbury Cleaners,43.685924,-79.376304,bar,24
2,Summerhill Ave / Maclennan Ave,Rosedale's Finest,43.685924,-79.376304,bar,27
3,Summerhill Ave / Maclennan Ave,Rosedale 1 Hour Cleaners,43.685924,-79.376304,bar,39
4,Summerhill Ave / Maclennan Ave,Dolce Bakery,43.685924,-79.376304,bar,38


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [82]:
# Yelp endpoint for searching places
yelp_url = 'https://api.yelp.com/v3/businesses/search'
y_headers = {
    "Authorization": f"Bearer {YELP_API_KEY}",  # Use Bearer token for authentication
}

def yelp_places(latitude, longitude, query, radius=1000):
    params = {
        "term": query,  # Use 'term' instead of 'query'
        "latitude": latitude,
        "longitude": longitude,
        "radius": radius,
        "limit": 10  # Limit to 10 results
    }
    
    response = requests.get(yelp_url, headers=y_headers, params=params)
    return response.json()  # Return JSON data if the request is successful


In [93]:
# Using Lat Long of Toronto
y_latitude = 43.6532
y_longitude = -79.3832


y_restaurants = yelp_places(y_latitude, y_longitude, "restaurant")
y_results = y_restaurants.get('businesses', [])


y_restaurants_df = pd.DataFrame(y_results)
y_restaurants_df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,business_hours,attributes,price
0,lwYtNxviXQMP0k8ucLfaWQ,edna-vita-toronto,Edna + Vita,https://s3-media1.fl.yelpcdn.com/bphoto/ot2yoQ...,False,https://www.yelp.com/biz/edna-vita-toronto?adj...,16,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...",4.4,"{'latitude': 43.64940455, 'longitude': -79.382...",[],"{'address1': '77 Adelaide St W', 'address2': '...",+14375626099,+1 437-562-6099,428.336205,[],"{'business_temp_closed': None, 'menu_url': Non...",NaN
1,r_BrIgzYcwo1NAuG9dLbpg,pai-northern-thai-kitchen-toronto-5,Pai Northern Thai Kitchen,https://s3-media4.fl.yelpcdn.com/bphoto/uAnTcm...,False,https://www.yelp.com/biz/pai-northern-thai-kit...,3740,"[{'alias': 'thai', 'title': 'Thai'}]",4.4,"{'latitude': 43.64784, 'longitude': -79.38872}",[],"{'address1': '18 Duncan Street', 'address2': '...",+14169014724,+1 416-901-4724,739.273351,"[{'open': [{'is_overnight': False, 'start': '1...","{'business_temp_closed': None, 'menu_url': Non...",$$
2,qekAZVcOoJjMBnN5rJ_4fQ,the-rabbit-hole-toronto,The Rabbit Hole,https://s3-media3.fl.yelpcdn.com/bphoto/WX1mq-...,False,https://www.yelp.com/biz/the-rabbit-hole-toron...,46,"[{'alias': 'gastropubs', 'title': 'Gastropubs'}]",4.2,"{'latitude': 43.64995, 'longitude': -79.37976}",[],"{'address1': '21 Adelaide Street W', 'address2...",+16474806963,+1 647-480-6963,451.023940,"[{'open': [{'is_overnight': False, 'start': '1...","{'business_temp_closed': None, 'menu_url': 'ht...",NaN
3,mZRKH9ngRY92bI_irrHq6w,richmond-station-toronto,Richmond Station,https://s3-media3.fl.yelpcdn.com/bphoto/agCQCt...,False,https://www.yelp.com/biz/richmond-station-toro...,948,"[{'alias': 'newcanadian', 'title': 'Canadian (...",4.4,"{'latitude': 43.6513729, 'longitude': -79.379285}",[],"{'address1': '1 Richmond Street W', 'address2'...",+16477481444,+1 647-748-1444,374.815589,"[{'open': [{'is_overnight': False, 'start': '1...","{'business_temp_closed': None, 'menu_url': 'ht...",$$$
4,cbAVxQjvujfZU0RvqAmtzg,mogouyan-hand-pulled-noodles-toronto,Mogouyan Hand-Pulled Noodles,https://s3-media4.fl.yelpcdn.com/bphoto/HuptZw...,False,https://www.yelp.com/biz/mogouyan-hand-pulled-...,110,"[{'alias': 'noodles', 'title': 'Noodles'}, {'a...",4.5,"{'latitude': 43.65573, 'longitude': -79.38449}",[],"{'address1': '124 Dundas St W', 'address2': ''...",,,295.384358,"[{'open': [{'is_overnight': False, 'start': '1...","{'business_temp_closed': None, 'menu_url': Non...",$$


In [85]:
y_restaurants_df.dtypes

id                 object
alias              object
name               object
image_url          object
is_closed            bool
url                object
review_count        int64
categories         object
rating            float64
coordinates        object
transactions       object
location           object
phone              object
display_phone      object
distance          float64
business_hours     object
attributes         object
price              object
dtype: object

We get Rating and Review Count with YELP API, along with many more information, which we didn't get with FOURSQUARE API.

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [90]:
# Starting with an empty list
yelp_poi_data = []

# Selecting first 5 stations from the df_bike
yelp_selected_stations = df_bike.head(5)

# Iterate through each bike station in df_bike dataframe from city_bikes
for index, row in yelp_selected_stations.iterrows():
    latitude = row['latitude'] 
    longitude = row['longitude']
    station_name = row['station_name']
    
    # Query for restaurants, bars, and banks
    for query in ["restaurant", "bar", "bank"]:
        pois = yelp_places(latitude, longitude, query, radius=50)
        
        if pois and 'businesses' in pois:  # Yelp's results are in 'businesses'
            for place in pois['businesses']:
                distance = place.get('distance', None)
                rating = place.get('rating', None)
                review_count = place.get('review_count', None)
                
                poi_info = {
                    'station_name': station_name,
                    'name': place.get('name', ''),
                    'latitude': latitude,
                    'longitude': longitude,
                    'category': query,
                    'distance': distance,
                    'rating': rating,  # Adding rating
                    'review_count': review_count  # Adding review count
                }
                yelp_poi_data.append(poi_info)  # Add to the empty list


Put your parsed results into a DataFrame

In [89]:
# Create a df from the data
yelp_pois_df = pd.DataFrame(yelp_poi_data)

# Check the first few rows of the new df
yelp_pois_df.head()

,station_name,name,latitude,longitude,category,distance,rating,review_count
0,Summerhill Ave / Maclennan Ave,Dolce Bakery,43.685924,-79.376304,restaurant,25.138820,4.3,10
1,Summerhill Ave / Maclennan Ave,Td Bank Financial Group,43.685924,-79.376304,bank,25.988849,0.0,0
2,Navy Wharf Crt / Bremner Blvd,Kibo Sushi,43.640722,-79.391051,restaurant,24.139939,4.0,5
3,Navy Wharf Crt / Bremner Blvd,Mr. Greek,43.640722,-79.391051,restaurant,26.719331,2.4,52
4,Navy Wharf Crt / Bremner Blvd,St. Louis Bar & Grill,43.640722,-79.391051,restaurant,58.385019,2.4,116


# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Yelp provides more complete data compared to Foursquare. Because -

1. Yelp offers rating and review_count, which are useful for our analysis purpose. Foursquare lacks these, making it harder to assess quality.

2. Yelp includes 18 fields, such as price, business_hours, and transactions. Foursquare has only 11, missing these useful details.

3. Yelp's additional data points make it more practical for decision-making, while Foursquare focuses on basic location information.

Yelp is more comprehensive, offering richer data for evaluating businesses, while Foursquare is more limited in scope.

Get the top 10 restaurants according to their rating

In [95]:
def yelp_top_restaurants(latitude, longitude, query="restaurant", radius=1000):
    # Define parameters for the Yelp API call
    params = {
        "term": query,  # Search term
        "latitude": latitude,
        "longitude": longitude,
        "radius": radius,
        "limit": 50,  # Get up to 50 results
        "sort_by": "rating"  # Sort by rating to get the highest-rated ones
    }
    
    response = requests.get(yelp_url, headers=y_headers, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        return None

In [99]:
# Using Lat Long of Toronto as an example
latitude = 43.6532
longitude = -79.3832

# Query Yelp for top restaurants
y_restaurants = yelp_top_restaurants(latitude, longitude, "restaurant")

# Extract the businesses data
if y_restaurants and 'businesses' in y_restaurants:
    y_restaurants_df = pd.DataFrame(y_restaurants['businesses'])
    
    # Sort by rating
    y_restaurants_df_sorted = y_restaurants_df.sort_values(by="rating", ascending=False)
    
    # Select top 10
    top_10_restaurants = y_restaurants_df_sorted.head(10)
    
    # Display the top 10
    print(top_10_restaurants[['name', 'rating', 'review_count']])

                   name  rating  review_count
0                 Mazeh     5.0             7
1                Adobar     5.0             5
2            Naan Kabob     5.0             5
18             La Crêpe     5.0             4
19        Ciao Pizzeria     5.0             4
3         Cucini Xpress     4.9             7
6        Firefly Burger     4.9             7
5   Trinity Square Cafe     4.9             7
4       Shawarma Royale     4.9             7
7           Dahu Hotpot     4.8             6


In [100]:
y_restaurants_df_sorted.head(10)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,business_hours,attributes,price
0,C-UU3BAhbP8hlYFozri2Yg,mazeh-toronto,Mazeh,https://s3-media3.fl.yelpcdn.com/bphoto/gwTsYO...,False,https://www.yelp.com/biz/mazeh-toronto?adjust_...,7,"[{'alias': 'lebanese', 'title': 'Lebanese'}, {...",5.0,"{'latitude': 43.65734, 'longitude': -79.38093}",[],"{'address1': '335 Yonge Street', 'address2': '...",+14164277366,+1 416-427-7366,504.631006,"[{'open': [{'is_overnight': False, 'start': '1...","{'business_temp_closed': None, 'menu_url': Non...",NaN
1,-ZXrJ65fAqIMYo98yzDkrw,adobar-toronto-3,Adobar,https://s3-media1.fl.yelpcdn.com/bphoto/kf9OQS...,False,https://www.yelp.com/biz/adobar-toronto-3?adju...,5,"[{'alias': 'filipino', 'title': 'Filipino'}, {...",5.0,"{'latitude': 43.6573225, 'longitude': -79.3738...",[],"{'address1': '200 Dundas Street E', 'address2'...",,,878.504083,[],"{'business_temp_closed': None, 'menu_url': Non...",NaN
2,S-jKUDEKOmaOpgdU32gNWA,naan-kabob-toronto-3,Naan Kabob,https://s3-media2.fl.yelpcdn.com/bphoto/CpLxuz...,False,https://www.yelp.com/biz/naan-kabob-toronto-3?...,5,"[{'alias': 'kebab', 'title': 'Kebab'}, {'alias...",5.0,"{'latitude': 43.65035552072757, 'longitude': -...",[],"{'address1': '240 Queen Street W', 'address2':...",+14166720358,+1 416-672-0358,680.093613,"[{'open': [{'is_overnight': False, 'start': '1...","{'business_temp_closed': None, 'menu_url': Non...",NaN
18,eiegu2iNZvlMcvxliVS4bA,la-crêpe-toronto,La Crêpe,https://s3-media1.fl.yelpcdn.com/bphoto/xDm5p4...,False,https://www.yelp.com/biz/la-cr%C3%AApe-toronto...,4,"[{'alias': 'creperies', 'title': 'Creperies'}]",5.0,"{'latitude': 43.64913, 'longitude': -79.39162}",[],"{'address1': '300 Richmond Street W', 'address...",+14169799885,+1 416-979-9885,814.728585,[],"{'business_temp_closed': None, 'menu_url': Non...",NaN
19,vyCRNs_5AkO68vpzwUttvg,ciao-pizzeria-toronto,Ciao Pizzeria,https://s3-media1.fl.yelpcdn.com/bphoto/gNl6eF...,False,https://www.yelp.com/biz/ciao-pizzeria-toronto...,4,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,"{'latitude': 43.65423442143131, 'longitude': -...",[],"{'address1': '334 Dundas Street W', 'address2'...",,,757.933577,"[{'open': [{'is_overnight': False, 'start': '1...","{'business_temp_closed': None, 'menu_url': Non...",NaN
3,hARWodRuAT1inXSbPMLeoQ,cucini-xpress-toronto,Cucini Xpress,https://s3-media4.fl.yelpcdn.com/bphoto/ae9qMz...,False,https://www.yelp.com/biz/cucini-xpress-toronto...,7,"[{'alias': 'indpak', 'title': 'Indian'}, {'ali...",4.9,"{'latitude': 43.6587182, 'longitude': -79.382272}",[],"{'address1': '372 Yonge Street', 'address2': '...",+14168271815,+1 416-827-1815,618.120274,"[{'open': [{'is_overnight': False, 'start': '1...","{'business_temp_closed': None, 'menu_url': Non...",NaN
6,d3ArU0Y0UO556bFocZp_2g,firefly-burger-toronto,Firefly Burger,https://s3-media4.fl.yelpcdn.com/bphoto/gFYM5j...,False,https://www.yelp.com/biz/firefly-burger-toront...,7,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.9,"{'latitude': 43.65796885488649, 'longitude': -...",[],"{'address1': '355 Yonge Street', 'address2': '...",+16472313473,+1 647-231-3473,550.469831,[],"{'business_temp_closed': None, 'menu_url': Non...",NaN
5,PT6tAoQxtCqsGc7r4nEXLQ,trinity-square-cafe-toronto-2,Trinity Square Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/Bczkp7...,False,https://www.yelp.com/biz/trinity-square-cafe-t...,7,"[{'alias': 'cafes', 'title': 'Cafes'}]",4.9,"{'latitude': 43.65392764383084, 'longitude': -...",[],"{'address1': '56 Queen Street E', 'address2': ...",+16472230975,+1 647-223-0975,526.173785,"[{'open': [{'is_overnight': False, 'start': '1...","{'business_temp_closed': None, 'menu_url': 'ht...",NaN
4,vibZWejr0SZH_YrQ_kJ7Ow,shawarma-royale-toronto-4,Shawarma Royale,https://s3-media1.fl.yelpcdn.com/bphoto/hrnuHW...,False,https://www.yelp.com/biz/shawarma-royale-toron...,7,"[{'alias': 'mideastern', 'title': 'Middle East...",4.9,"{'latitu